# Cluster genes along pseudotime 

Summary of time series clustering in R  
https://journal.r-project.org/archive/2016/RJ-2016-058/index.html  
https://cran.r-project.org/web/packages/dtwclust/vignettes/dtwclust.pdf  

### Distance meassures 

Shape meassures: Lock-step (n=m) and elastic (n!=m)  
Feature based: Fourier or wavelet coefficients, autocorrelation values, etc.  
Structure-based: (i) model-based approaches, where a model is fit to each series and the comparison is made between models, and (ii) complexity-based models, where the similarity between two series is measured based on the quantity of shared information.  
Prediction-based distances analyze the similarity of the forecasts obtained for different time series.  

In [1]:
options(warn=-1)
options(dplyr.summarise.inform=FALSE)

In [2]:
library_load <- suppressMessages(
    
    list(
        
        # parallelDist
        library(parallelDist), 
        
        # TradeSeq
        library(tradeSeq), 
        
        # Seurat 
        library(Seurat), 
        
        # GSEA
        library(fgsea), 
        library(msigdbr), 
        
        # Data 
        library(tidyverse), 
        
        # Plotting 
        library(ggplot2), 
        library(patchwork), 
        library(ggplotify),
        library(ComplexHeatmap),
        library(circlize)
        
    )
)

In [3]:
random_seed <- 42
set.seed(random_seed)

In [4]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [5]:
# Source files
source("plotting_global.R")
source("bin/tradeseq_pp.R")
source("bin/tradeseq_pl.R")

In [6]:
# Plotting Theme
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter settings 

In [7]:
tradeseq_res_file <- "result/lineage/tradeseq_res_2.rds"

In [ ]:
contrast_vec <- list(

    c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1"), 
    c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D3"), 
    c("IFNAR_fl_CpG_D1", "IFNAR_fl_CpG_D3"), 
    c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_CpG_D1"), 
    c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_CpG_D3"), 
    c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D3"), 
    c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0"), 
    c("IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1"), 
    c("IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3")
    
)

# Import data 

In [ ]:
tradeseq_res <- readRDS(tradeseq_res_file)

In [ ]:
fitgam <- tradeseq_res[[1]]

In [ ]:
condition_levels <- levels(fitgam@colData$tradeSeq$conditions)

# Condition test 

In [ ]:
condition_res <- conditionTest(fitgam, l2fc=0.00, pairwise=TRUE)

In [ ]:
ptpg <- lapply(contrast_vec, function(x) {

    message(paste0(x[1], " vs ", x[2]))

    # Get condition fit
    idx <- paste0(which(condition_levels %in% x), collapse="vs")
    ptpg_i <- condition_res[, grepl(idx, colnames(condition_res))] %>% tibble::rownames_to_column("gene")
    colnames(ptpg_i) <- gsub(paste0("_conds", idx), "", colnames(ptpg_i))

    # Clean up data 
    ptpg_i <- ptpg_i %>% dplyr::rename(p_val_wald=pvalue, wald_stat=waldStat) %>% dplyr::mutate(p_val_wald=ifelse(is.na(p_val_wald), 1, p_val_wald), wald_stat=ifelse(is.na(wald_stat), 0, wald_stat))
    
    # Condition fit
    ptpg_i <- ptpg_i  %>%
        dplyr::mutate(p_val_adj_wald=p.adjust(p_val_wald, method="BH")) %>% 
        dplyr::mutate(ptpg_wald=ifelse(p_val_adj_wald<=0.01, TRUE, FALSE)) 

    print(table(ptpg_i$ptpg_wald))
    
    # Condition DEA 
    mat_1 <- pt_diff_smooth(fitgam, genes=ptpg_i[ptpg_i$ptpg_wald, ]$gene, condition_qry=x[2], condition_ref=x[1])
    mat_2 <- pt_diff_wilcox(fitgam, genes=ptpg_i[ptpg_i$ptpg_wald, ]$gene, condition_qry=x[2], condition_ref=x[1])

    # Check for any log2FC in any bin 
    sig_check_1 <- mat_1 %>% dplyr::filter(abs(diff_smooth)>=0.50) %>% dplyr::pull(gene) %>% unique()
    sig_check_2 <- mat_1 %>% group_by(gene) %>% dplyr::summarise(diff_smooth=sum(abs(diff_smooth))) %>% dplyr::filter(diff_smooth>=1.00) %>% dplyr::pull(gene) %>% unique()
    sig_check <- union(sig_check_1, sig_check_2)
    
    # Combine mat 
    mat <- dplyr::left_join(mat_1 %>% dplyr::select(gene, pt_bin, diff_smooth), mat_2 %>% dplyr::select(gene, pt_bin, log2FC_wilcox, p_val_adj_wilcox), by=join_by(gene, pt_bin)) %>% na.omit()
        
    ptpg_class <- mat %>% dplyr::arrange(gene, pt_bin) %>% 
        dplyr::filter((abs(log2FC_wilcox)>=0.25 | abs(diff_smooth)>=0.25) & p_val_adj_wilcox<=0.05 & sign(diff_smooth)==sign(log2FC_wilcox)) %>% 
        dplyr::group_by(gene) %>%
        dplyr::mutate(ptpg_class=ifelse(all(sign(log2FC_wilcox)==+1), x[2], ifelse(all(sign(log2FC_wilcox)==-1), x[1], "Perturbed"))) %>% 
        dplyr::select(gene, ptpg_class) %>% dplyr::distinct()
    
    # Set PTPG class
    ptpg_i <- dplyr::left_join(ptpg_i, ptpg_class, by=join_by(gene)) %>% 
        dplyr::mutate(ptpg_class=ifelse(is.na(ptpg_class) & ptpg_wald, "Perturbed", ptpg_class)) %>% 
        dplyr::mutate(ptpg_class=ifelse(!(gene %in% sig_check), "Canonical", ptpg_class)) 

    ptpg_i <- ptpg_i %>% dplyr::mutate(contrast=paste0(x, collapse=":"))

    print(table(ptpg_i$ptpg_class))

    return(ptpg_i)

}
      )

# Save results 

In [ ]:
names(ptpg) <- lapply(contrast_vec, function(x) paste0(x, collapse=":"))
tradeseq_res[["ptpg"]] <- purrr::imap_dfr(ptpg, ~ mutate(.x, contrast=.y))

In [12]:
tradeseq_res <- readRDS(tradeseq_res_file)
res <- tradeseq_res[["ptpg"]]

In [15]:
res <- split(res, f=res$contrast)

In [23]:
names(res) <- c(

    "ctl_vs_d1", "ctl_vs_d3", "ctl_vs_ctl_ko", 
    "ctl_vs_d1_ko", "ctl_vs_3_ko", "d1_vs_d3", 
    "d1_vs_d1_ko", "d3_vs_d3_ko", "d1_ko_vs_d3_ko"

    
)

In [24]:
openxlsx::write.xlsx(res, "result/lineage/ptpg_cg_ko.xlsx")

In [ ]:
saveRDS(tradeseq_res, tradeseq_res_file)

# SessionInfo 

In [ ]:
sessionInfo()